<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: Лисицын Сергей
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 9. Разреженные данные, онлайн-обучение</center>

## <center>Часть 4. Применение vowpal wabbit для соревнования Avazu</center>

Одной из важнейших задач для компаний, связанных с интернет-рекламой, является предсказание CTR (Click-Through Rate), величины, соответствующей вероятности клика некоторого пользователя по некоторому баннеру. В Kaggle было проведено два соревнования по предсказанию CTR, одно из которых мы рассмотрим в данной части занятия.

Соревнование, о котором идет речь, называется Click-Through Rate Prediction и представлено компанией Avazu. Дополнительную информацию можно найти по ссылке: https://www.kaggle.com/c/avazu-ctr-prediction. Размер данных не позволяет поместить их в репозиторий, поэтому для воспроизведения дальнейших операций вам нужно сохранить выборку по пути `../../data/avazu_train.csv`.

Ознакомимся с форматом данных:

In [3]:
with open('../../data/avazu_train.csv') as train_file:
    for _ in range(6):
        print(train_file.readline().strip())

id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722,0,35,-1,79
10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,96809ac8,711ee120,1,0,15704,320,50,1722,0,35,100084,79
10000371904215119486,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,b3cf8def,8a4875bd,1,0,15704,320,50,1722,0,35,100084,79
10000640724480838376,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8275b8f,6332421a,1,0,15706,320,50,1722,0,35,100084,79
10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,a99f214a,9644d0bf,779d90c2,1,0,18993,320,50,2161,0,35,-1,157


Одной из особенностей работы с такими данными является то, что полная их загрузка в память может быть невозможна. Поэтому в этой части занятия мы не будем пользоваться pandas и будем работать на более низком уровне в "потоковом" режиме, никогда не загружая весь файл в память.

Ознакомимся с атрибутами:

- **id** идентификатор события. Очевидно, что идентификатор не является обобщающим признаком, поэтому будет исключен
- **click** произошел ли клик? 0 или 1
- **hour** час дня, в котороый произошел показ
- **C1** анонимизированный атрибут
- **banner_pos** положение баннера, дискретная величина
- **site_id** идентификатор сайта, на котором произошел показ
- **site_domain** идентификатор домена сайта, на котором произошел показ
- **site_category** идентификатор категории сайта, на котором произошел показ
- **app_id** аналогично, идентификатор приложения
- **app_domain** идентификатор домена приложения
- **app_category** идентификатор категории приложения
- **device_id** идентификатор устройства. Является уникальным среди пользователей признаком, что может привести к переобучению, будет исключен
- **device_ip** IP-адрес устройства. Чуть менее уникален, чем идентификатор 
- **device_model** модель устройства
- **device_type** тип устройства
- **device_conn_type** тип соединения устройства с интернетом
- **C14,C15,C16,C17,C18,C19,C20,C21** анонимизированный атрибут

Таким образом, в специальной обработке нуждаются click (как непосредственно метка класса), и исключаемые нами id и device_id. 

Будем считывать данные с помощью библиотеки csv в виде dict:

In [12]:
import csv
with open('../../data/avazu_train.csv') as train_file:
    reader = csv.reader(train_file)
    header = next(reader)
    print('Attributes are', header)
    some_example = dict(zip(header, next(reader)))
    
some_example

Attributes are ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']


{'C1': '1005',
 'C14': '15706',
 'C15': '320',
 'C16': '50',
 'C17': '1722',
 'C18': '0',
 'C19': '35',
 'C20': '-1',
 'C21': '79',
 'app_category': '07d7df22',
 'app_domain': '7801e8d9',
 'app_id': 'ecad2386',
 'banner_pos': '0',
 'click': '0',
 'device_conn_type': '2',
 'device_id': 'a99f214a',
 'device_ip': 'ddd2926e',
 'device_model': '44956a24',
 'device_type': '1',
 'hour': '14102100',
 'id': '1000009418151094273',
 'site_category': '28905ebd',
 'site_domain': 'f3845767',
 'site_id': '1fbe01fe'}

Реализуем функцию для преобразования в формат VW:

In [23]:
def vowpalize(row, include_label=False):
    result = []
    if include_label:
        result.append(row['click'])
        
    # weight
    result.append(str(1.0))
    result.append('|features')
        
    for k,v in row.items():
        if k in ('id', 'device_id', 'click'):
            continue
            
        result.append('{}={}'.format(k,v))
    return ' '.join(result) + '\n'
        
vowpalize(some_example)

'1.0 |features app_id=ecad2386 C1=1005 C21=79 device_model=44956a24 site_id=1fbe01fe C15=320 C20=-1 banner_pos=0 device_conn_type=2 C16=50 device_type=1 C17=1722 C18=0 site_domain=f3845767 app_category=07d7df22 site_category=28905ebd device_ip=ddd2926e hour=14102100 C14=15706 C19=35 app_domain=7801e8d9\n'

Теперь мы готовы преобразовать данные. Отнесем 10% данных в тестовую выборку:

In [29]:
import random

with open('../../data/avazu_train.csv') as train_file, \
     open('../../output/avazu_train.vw', 'w') as vw_train_file, \
     open('../../output/avazu_test.vw', 'w') as vw_test_file:
    reader = csv.reader(train_file)
    header = next(reader)
    for line in reader:
        example = dict(zip(header, line))
        if random.random() < 0.1:
            vw_test_file.write(vowpalize(example, include_label=False))
        else:
            vw_train_file.write(vowpalize(example))

KeyboardInterrupt: 

Обучим модель с логистической функцией потерь, подходящей для этой задачи:

! { ваш код здесь }

Применим модель на построенной тестовой выборке:

In [ ]:
! { ваш код здесь } 

Оценим качество работы классификатора:

In [ ]:
{ ваш код здесь }